# Machine Learning
## Lab02 - Let's PyTorch, Light the Flame of Learning Machine Learning

Nevermind if you do not understand YET!

OBjective: This lab is just to let you have a quick hands-on exposure to another example of more advanced **neural network** using the high-level library called **PyTorch**. Do not worry if you do not understand WHY it works, just play around with it to have an idea HOW it works.

## Mount Google Drive

In [ ]:
# This is needed if you need to read data from your Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Import Packages

In [ ]:
# If run from the desktop, use Anaconda to install the below:
#    conda install pytorch torchvision

# Setting random seeds to ensure we have the same results each time we run the code,
#    this is not guaranteed across PyTorch releases.

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from timeit import default_timer as timer

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Load Training Data of FMNIST Datasets from TorchVision

In [ ]:
from torchvision import datasets, transforms

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training data and load training data
training_data = datasets.FashionMNIST('~/.pytorch/FMNIST/',
                                      download = True, train = True,
                                      transform = transform
                                     )

## Understanding and Explore the Training Data


In [ ]:
print("Size of training_data =", len(training_data))
print("type(training_data[0][0]) =", type(training_data[0][0]))   # image
print("type(training_data[0][1]) =", type(training_data[0][1]))   # label (integer)

print("training_data[0][0].size() =", training_data[0][0].size())

In [ ]:
FMNIST_labels_map = ['T-shirt','Trouser','Pullover','Dress','Coat',
                     'Sandal','Shirt','Sneaker','Bag','Ankle Boot']

def get_FMNIST_label_name(label):
  label_name = FMNIST_labels_map[label]
  return label_name

def print_FMNIST_label_name(index, label):
  label_name = get_FMNIST_label_name(label)
  print(f"Label name for sample with index {index} = {label_name}")

def show_FMNIST_image(image):
  plt.imshow(image.squeeze(), cmap = 'gray')

In [ ]:
# TODO: try different number,
#       must be from 0 to 59999 for traing_data and from 0 to 9999 for testing_data
sample_image_index = 17

sample_image, sample_image_label = training_data[sample_image_index]

print_FMNIST_label_name(sample_image_index, sample_image_label)
print()
show_FMNIST_image(sample_image)

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size = (1,)).item()
    img, label = training_data[sample_idx]

    figure.add_subplot(rows, cols, i)

    label_name = get_FMNIST_label_name(label)
    plt.title(label_name)

    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Understanding and Explore the DataLoader

In [ ]:
# Prepare Training Data with DataLoaders
train_dataloader = torch.utils.data.DataLoader(training_data,
                                               batch_size = 64,
                                               shuffle = False
                                              )

# Load one batch of training data
training_batch_images, training_batch_labels = next(iter(train_dataloader))

print("len(training_batch_images) =", len(training_batch_images))
print("len(training_batch_labels) =", len(training_batch_labels))

print("type(training_batch_images) =", type(training_batch_images))
print("type(training_batch_labels) =", type(training_batch_labels))

print("training_batch_images.shape =", training_batch_images.shape)
print("training_batch_labels.shape =", training_batch_labels.shape)

In [ ]:
# TODO: try different number,
#       must be from 0 to 59999 for traing_data and from 0 to 9999 for testing_data
sample_image_index = 17

sample_image       = training_batch_images[sample_image_index]
sample_image_label = training_batch_labels[sample_image_index]

print_FMNIST_label_name(sample_image_index, sample_image_label)
print()
show_FMNIST_image(sample_image)

## Discussion

Q: What is the size of an image?  
A:


## Build a PyTorch `Sequential` Model

> Indented block



In [ ]:
def create_sequential_model():
  return nn.Sequential(nn.Flatten(start_dim = 1, end_dim = -1),
                       nn.Linear(784, 128),
                       nn.ReLU(),
                       nn.Linear(128,  64),
                       nn.ReLU(),
                       nn.Linear( 64,  10),
                       nn.LogSoftmax(dim = 1))

## Build a PyTorch `nn.Module` Model

- This is alternative to the PyTorch `Sequential` model
- Sequential is a subclass of `nn.Module`

In [ ]:
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128,  64)
    self.fc3 = nn.Linear( 64,  10)

  def forward(self, x):
    x = x.view(x.shape[0], -1)

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim = 1)

    return x

## Define the Training Function

In [ ]:
def train(model, batch_size):

  print("Training: ", end = "")

  criterion = nn.NLLLoss()
  optimizer = optim.SGD(model.parameters(), lr = 0.01)

  train_dataloader = torch.utils.data.DataLoader(training_data,
                                                 batch_size = batch_size,
                                                 shuffle = False
                                                )

  total_batches = int(len(training_data) / batch_size + 0.5)  # round up to integer

  start = timer()

  cum_loss = 0

  for batch_num, (images, labels) in enumerate(train_dataloader, 1):
      optimizer.zero_grad()
      output = model(images)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()

      print(f"{batch_num}/{total_batches}:{loss.item()} ", end = "")

      cum_loss += loss.item()

  training_loss = cum_loss/len(train_dataloader)

  time_taken = timer() - start
  print()
  print(f"Training loss = {training_loss} ", end = "")
  print("Time taken =", time_taken)

  return training_loss

## Train a PyTorch `Sequential` Model

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

sequential_model = create_sequential_model()

train(model = sequential_model, batch_size = 64)
train(model = sequential_model, batch_size = 64)

## Train a PyTorch `nn.Module` Model

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

nn_module_model = FMNIST()

train(model = nn_module_model, batch_size = 64)
train(model = nn_module_model, batch_size = 64)

## Load, Understand and Understand the Test Data of FMNIST Datasets from TorchVision

In [ ]:
mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST test dataset and load test data
testing_data = datasets.FashionMNIST('~/.pytorch/FMNIST/',
                                     download = True, train = False,
                                     transform = transform
                                    )

In [ ]:
print("Size of testing_data =", len(testing_data))
print("type(testing_data[0][0]) =", type(testing_data[0][0]))   # image
print("type(testing_data[0][1]) =", type(testing_data[0][1]))   # label (integer)

print("testing_data[0][0].size() =", testing_data[0][0].size())

## Test the Model with a Sample Test Data

In [ ]:
# TODO: try different number,
#       must be from 0 to 59999 for traing_data and from 0 to 9999 for testing_data
sample_image_index = 127

sample_image, sample_image_label = testing_data[sample_image_index]

print_FMNIST_label_name(sample_image_index, sample_image_label)
print()
show_FMNIST_image(sample_image)

In [ ]:
with torch.no_grad(): # perform inference/evaluation without computing gradients or storing intermediate values
  logps = sequential_model(sample_image) # log probabilities

ps = torch.exp(logps) # probabilities

print("logps =", logps)
print("ps =", ps)

nps = ps.numpy()[0] # in numpy list
print("nps =", nps)

In [ ]:
plt.xticks(np.arange(10), labels = FMNIST_labels_map, rotation = 'vertical')

plt.bar(np.arange(10), nps)

## Test the Model with All Test Data

In [ ]:
def test(model, batch_size):

  print("Testing: ", end = "")

  criterion = nn.NLLLoss()

  test_dataloader = torch.utils.data.DataLoader(testing_data,
                                                batch_size = batch_size,
                                                shuffle = False
                                               )

  start = timer()

  cum_loss = 0

  for batch_num, (images, labels) in enumerate(test_dataloader, 1):
    output = model(images)
    loss = criterion(output, labels)

    cum_loss += loss.item()

  testing_loss = cum_loss/len(test_dataloader)

  time_taken = timer() - start

  print(f"Testing loss = {testing_loss} ", end = "")
  print("Time taken =", time_taken)

  return testing_loss

In [ ]:
test(model = sequential_model, batch_size = 64)

## Training and Testing

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

sequential_model = create_sequential_model()

num_epochs = 20
batch_size = 64

training_losses = []
testing_losses = []

for i in range(1, num_epochs + 1):

  print(f"Epoch {i}/{num_epochs} => ")

  training_loss = train(model = sequential_model, batch_size = batch_size)
  testing_loss  = test( model = sequential_model, batch_size = batch_size)

  training_losses.append(training_loss)
  testing_losses.append(testing_loss)

  print(f"Epoch {i}/{num_epochs} training loss = {training_loss} testing loss = {testing_loss}")

print(training_losses)
print(testing_losses)

In [ ]:
x = list( range(1, num_epochs + 1) )
x_range = (1, num_epochs + 1)

plt.xticks(range(1, num_epochs + 1, 1))

plt.plot(x, training_losses, color = "r", label = "Training Losses")
plt.plot(x, testing_losses,  color = "b", label = "Testing Losses")

plt.locator_params(axis='y', nbins=10)

plt.xlim(x_range)

plt.xlabel("Epoch")
plt.ylabel("LOsses")
plt.title("Training & Testing Losses")
plt.legend()
plt.show()

## Working with GPUs

In Google Colab, go to the menu "Runtime -> Change run time type" to set the "Hardware accelerator" to "GPU"

In [ ]:
# NVIDIA (R) Cuda compiler driver
!nvcc --version

In [ ]:
# NVIDIA System Management Interface (nvidia-smi)
!nvidia-smi

In [ ]:
# Refer to https://pytorch.org/get-started/locally/
# conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def train(model, batch_size, use_gpu = False):

  print("use_gpu =", use_gpu)
  print("Training: ", end = "")

  criterion = nn.NLLLoss()
  optimizer = optim.SGD(model.parameters(), lr = 0.01)

  train_dataloader = torch.utils.data.DataLoader(training_data,
                                                 batch_size = batch_size,
                                                 shuffle = False
                                                )

  total_batches = int(len(training_data) / batch_size + 0.5)  # round up to integer

  start = timer()

  cum_loss = 0

  for batch_num, (images, labels) in enumerate(train_dataloader, 1):
      optimizer.zero_grad()

      if (use_gpu == True):
        # Move data to GPU
        images = images.to(device)
        labels = labels.to(device)

      output = model(images)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()

      print(f"{batch_num}/{total_batches}:{loss.item()} ", end = "")

      cum_loss += loss.item()

  training_loss = cum_loss/len(train_dataloader)

  time_taken = timer() - start
  print()
  print(f"Training loss = {training_loss} ", end = "")
  print("Time taken =", time_taken)

  return training_loss

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

sequential_model_gpu = create_sequential_model()

# Move model to GPU
sequential_model_gpu.to(device)

# Train model in GPU
train(model = sequential_model_gpu, batch_size = 64, use_gpu = True)
train(model = sequential_model_gpu, batch_size = 64, use_gpu = True)

In [ ]:
# TODO: try different number,
#       must be from 0 to 59999 for traing_data and from 0 to 9999 for testing_data
sample_image_index = 127

sample_image, sample_image_label = testing_data[sample_image_index]

print_FMNIST_label_name(sample_image_index, sample_image_label)
print()
show_FMNIST_image(sample_image)

# Move model back to CPU
sequential_model_gpu.to("cpu")

with torch.no_grad(): # perform inference/evaluation without computing gradients or storing intermediate values
  logps = sequential_model_gpu(sample_image) # log probabilities

ps = torch.exp(logps) # probabilities

print("logps =", logps)
print("ps =", ps)

nps = ps.numpy()[0] # in numpy list
print("nps =", nps)

plt.xticks(np.arange(10), labels = FMNIST_labels_map, rotation = 'vertical')

plt.bar(np.arange(10), nps)


## Step Through Sequential Model

In [ ]:
def print_weights(model, index):
  print(f"model[{index}].weight =>", model[index].weight)
  print(f"model[{index}].weight.grad =>", model[index].weight.grad)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

train_dataloader = torch.utils.data.DataLoader(training_data,
                                               batch_size = 64,
                                               shuffle = False
                                              )

# Load one batch of training data
images, labels = next(iter(train_dataloader))

print("images.shape =", images.shape)

In [ ]:
mini_sequential_model = create_sequential_model()

criterion = nn.NLLLoss()
optimizer = optim.SGD(mini_sequential_model.parameters(), lr = 0.01)

print_weights(mini_sequential_model, 1)

In [ ]:
output = mini_sequential_model(images)
loss = criterion(output, labels)
loss.backward()

print(f"loss.item() = {loss.item()} ")

print_weights(mini_sequential_model, 1)

In [ ]:
optimizer.step()

print_weights(mini_sequential_model, 1)

In [ ]:
optimizer.zero_grad()

print_weights(mini_sequential_model, 1)

In [ ]:
output = mini_sequential_model(images)
loss = criterion(output, labels)
loss.backward()
optimizer.step()

print(f"loss.item() = {loss.item()} ")
print_weights(mini_sequential_model, 1)

In [ ]:
optimizer.zero_grad()
output = mini_sequential_model(images)
loss = criterion(output, labels)
loss.backward()
optimizer.step()

print(f"loss.item() = {loss.item()} ")
print_weights(mini_sequential_model, 1)

In [ ]:
print(mini_sequential_model[0])
print(mini_sequential_model[1])
print(mini_sequential_model[2])
print(mini_sequential_model[3])
print(mini_sequential_model[4])
print(mini_sequential_model[5])
print(mini_sequential_model[6])
